In [1]:
import torch
import numpy as np
import os
for key, value in os.environ.items():
    print(f"{key}: {value}")
os.environ["LD_LIBRARY_PATH"] = "/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/usr/local/cuda/bin:/usr/local/cuda/lib64:/home/nvan21/.mujoco/mujoco210/bin"

SHELL: /bin/bash
SLURM_JOB_USER: nvan21
SLURM_TASKS_PER_NODE: 4
SLURM_JOB_UID: 434058
HISTCONTROL: ignoredups
SLURM_TASK_PID: 1087574
CONDA_EXE: /work/flemingc/nvan21/miniconda3/bin/conda
_CE_M: 
SLURM_JOB_GPUS: 2
SLURM_LOCALID: 0
SLURM_SUBMIT_DIR: /work/flemingc/nvan21/projects
HOSTNAME: nova22-gpu-5
HISTSIZE: 1000
SLURMD_NODENAME: nova22-gpu-5
SLURM_JOB_START_TIME: 1721318139
HYDRA_LAUNCHER_EXTRA_ARGS: --external-launcher
_ModuleTable002_: cy9Db3JlIiwKfSwKc3lzdGVtQmFzZU1QQVRIID0gIi9ldGMvbW9kdWxlZmlsZXM6L3Vzci9zaGFyZS9tb2R1bGVmaWxlczovdXNyL3NoYXJlL21vZHVsZWZpbGVzL0xpbnV4Oi91c3Ivc2hhcmUvbW9kdWxlZmlsZXMvQ29yZTovdXNyL3NoYXJlL2xtb2QvbG1vZC9tb2R1bGVmaWxlcy9Db3JlIiwKfQo=
PTMPDIR: /mnt/job-temp/nvan21/5353820/ptmp
__LMOD_REF_COUNT_MODULEPATH: /opt/rit/singularity/modules:2;/opt/rit/el9/current/modules/lmod/linux-rhel9-x86_64/Core:2;/opt/hpc/modulefiles:2;/etc/modulefiles:1;/usr/share/modulefiles:1;/usr/share/modulefiles/Linux:1;/usr/share/modulefiles/Core:1;/usr/share/lmod/lmod/modulefiles/C

In [3]:
import iql_utils
from iql_utils import TrainConfig as TrainConfig

/work/flemingc/nvan21/miniconda3/envs/iros/lib/python3.10/site-packages/Cython/Distutils/old_build_ext.py:14: DeprecationWarning: dep_util is Deprecated. Use functions from setuptools instead.
  from distutils.dep_util import newer, newer_group


Exception: 
Missing path to your environment variable. 
Current values LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/nvan21/.mujoco/mujoco210/bin
Please add following line to .bashrc:
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/nvidia

In [3]:
model = iql_utils.GaussianPolicy(4, 2, max_action=float(4.99),)

In [4]:
device = 'cpu' if not torch.cuda.is_available() else 'cuda'
print(f"device: {device}")

device: cuda


In [5]:
iql_all_models = torch.load('/home/prajwal/workfolder/CleanOfflineRL/CORL/iros_double_pendulum/iql_double-pendulum-sac-v2_500000_final.pt',
                            map_location=torch.device(device))

In [6]:
model.load_state_dict(iql_all_models["state_dict"]["actor"])
state_mean = iql_all_models["state_mean"]
state_std = iql_all_models["state_std"]
model.to(device)
model.eval()

GaussianPolicy(
  (net): MLP(
    (net): Sequential(
      (0): Linear(in_features=4, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=2, bias=True)
      (5): Tanh()
    )
  )
)

In [8]:
state_mean, state_std

(array([ 3.10138527e+00, -1.25685267e-01,  3.17212890e-01, -2.20910353e-03]),
 array([0.43561072, 0.38527233, 1.89671584, 2.06232309]))

In [7]:

obs = np.array([0.1, 0.2, 0.3, 0.4])
obs = iql_utils.normalize_states(obs, state_mean, state_std)
print(obs)

[-6.89006299  0.84533781 -0.0090751   0.1950272 ]


In [10]:
model.act(obs, device=device)

array([-1.2969251,  0.       ], dtype=float32)